# Reference

**Library**

* [librosa](https://librosa.org/doc/main/feature.html)


**Tech Blog**

* [제 06절 아날로그 신호](https://m.blog.naver.com/gkenq/220679236344)
* [AI에게 어떻게 음성을 가르칠까?](https://tech.kakaoenterprise.com/66)
* [음성인식 입문하기](https://pyy0715.github.io/Audio/)
* [음악 데이터 - spectral_centroid, sepectral_rolloff](https://0equal2.tistory.com/144)
* [[Librosa] music/audio processing library Librosa 사용법 Tutorial - (3) Audio feature extraction](https://bo-10000.tistory.com/entry/Librosa-musicaudio-processing-library-Librosa-%EC%82%AC%EC%9A%A9%EB%B2%95-Tutorial-3-Audio-feature-extraction)

* [Tonnetz 음악이론을 딥러닝에 접목시키기](https://inspiringpeople.github.io/data%20analysis/tonnetz-dl/)


**You Tube**

* [DMQA, Introduction to Analysis for Sound Data](https://www.youtube.com/watch?v=1Hhj14QhkaE&t=718s)


**데이콘 코드 공유**
* [mel-spectrogram, mfcc 활용한 CNN ensemble + 5fold / public 0.98](https://dacon.io/en/codeshare/5153)

# Setting

## Library

In [1]:
import pandas as pd
import numpy as np

import librosa

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD, IncrementalPCA
from sklearn.decomposition import TruncatedSVD

import os
from tqdm.auto import tqdm
import random

In [2]:
import librosa
import librosa.display
import IPython.display as ipd

In [3]:
import warnings
warnings.filterwarnings(action='ignore') 

## Fixed Random Seed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Load Data Set

## Google Drive Mount

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Unzip File

In [6]:
!unzip -qq '/content/drive/MyDrive/머신러닝 엔지니어링/데이콘/기계 고장 진단/data/기계_고장.zip'

## Load Train / Test Set

In [7]:
df_train = pd.read_csv('./train.csv') # 모두 정상 Sample
df_test = pd.read_csv('./test.csv')

In [8]:
print(df_train.shape)
df_train.head()

(1279, 4)


,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0


In [9]:
print(df_test.shape)
df_test.head()

(1514, 3)


,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE
0,TEST_0000,./test/TEST_0000.wav,2
1,TEST_0001,./test/TEST_0001.wav,2
2,TEST_0002,./test/TEST_0002.wav,0
3,TEST_0003,./test/TEST_0003.wav,0
4,TEST_0004,./test/TEST_0004.wav,0


# Feature Engineering

## Scaling

In [10]:
def scaled_df(df_train, df_train_fan_type, df_test, df_test_fan_type, scaler):

  df_train_fan_type = df_train_fan_type[['FAN_TYPE']]

  df_train = pd.concat([
                        df_train.reset_index(drop=True),
                        df_train_fan_type.reset_index(drop=True)
                       ],
                       axis=1)
  
  df_test_fan_type = df_test_fan_type[['FAN_TYPE']]

  df_test = pd.concat([
                       df_test.reset_index(drop=True),
                       df_test_fan_type.reset_index(drop=True)
                      ],
                      axis=1)
  
  train_type_0 = df_train.loc[(df_train['FAN_TYPE']==0)]
  train_type_2 = df_train.loc[(df_train['FAN_TYPE']==2)]

  test_type_0 = df_test.loc[(df_test['FAN_TYPE']==0)]
  test_type_2 = df_test.loc[(df_test['FAN_TYPE']==2)]

  train_type_0.drop(columns='FAN_TYPE', inplace=True)
  train_type_2.drop(columns='FAN_TYPE', inplace=True)
  test_type_0.drop(columns='FAN_TYPE', inplace=True)
  test_type_2.drop(columns='FAN_TYPE', inplace=True)

  list_train_0_index = list(train_type_0.index)
  list_train_2_index = list(train_type_2.index)

  list_test_0_index = list(test_type_0.index)
  list_test_2_index = list(test_type_2.index)

  scaled_train_type_0 = scaler.fit_transform(train_type_0)
  scaled_test_type_0 = scaler.transform(test_type_0)

  scaled_train_type_2 = scaler.fit_transform(train_type_2)
  scaled_test_type_2 = scaler.transform(test_type_2)

  train_type_0 = pd.DataFrame(scaled_train_type_0)
  train_type_2 = pd.DataFrame(scaled_train_type_2)

  test_type_0 = pd.DataFrame(scaled_test_type_0)
  test_type_2 = pd.DataFrame(scaled_test_type_2)

  train_type_0.index = list_train_0_index
  train_type_2.index = list_train_2_index

  test_type_0.index = list_test_0_index
  test_type_2.index = list_test_2_index

  df_train = pd.concat([train_type_0, train_type_2], axis=0)
  df_test = pd.concat([test_type_0, test_type_2], axis=0)

  df_train.sort_index(inplace=True)
  df_test.sort_index(inplace=True)

  return df_train, df_test

## Feature Extraction

**Sound**

* 공기나 물 같은 매질의 진동을 통해 전달되는 종파
* 사람의 귀에 들려오는 소리는 공기로 전해오는 파동

**소리의 3요소**

* 세기 (소리의 크기)
* 높낮이 (소리의 높고 낮음)
* 음색 (소리의 색상)

**아날로그 신호**

* 자연계에 포함되어 있는 연속적인 파형
* 주기 신호와 비주기 신호로 분류할 수 있다

**주기 신호**

* 연속적으로 반복된 패턴으로 구성
* 사이클 (Cycle) - 하나의 완성된 패턴
* 정형파와 비정형파로 분류할 수 있다

**비주기 신호**

* 시간에 따라 반복된 패턴이나 사이클 없이 항상 변한다


**정형파**

1. 진폭 (Amplitude)

  - 신호의 크기나 또는 세기를 나타낸다
  - 신호의 높이
  - 단위 : V (volt)

2. 주기 (Period)와 주파수 (Frequency)

  - 주기 : 하나의 사이클을 완성하는데 필요한 시간
  - 주파수 : 주기의 역수, 1초에 완성되는 주기 횟수
  - 높은 주파수 : 짧은 기간내의 변화
  - 낮은 주파수 : 긴 기간에 걸친 변화
  - 단위 : HZ (cycle / second)

3. 위상 (phase)

  - 임의의 시간에서 반송파 사이클의 상대적인 위치
  - 시간 0에서 대한 파형의 상대적인 위치
  - 시간축을 따라 앞뒤로 이동될 수 있는 파형에서 그 이동된 양
  - 첫 사이클의 상태
  - 단위 : degree

4. 대역폭 (Bandwidth)

  - 복합신호의 대역폭은 신호에 포함된 최고 주파수와 최저 주파수 사이의 차이

**Sampling Rate**

* 소리를 컴퓨터에 입력시키기 위해 음파를 숫자로 표현할 필요가 있음

* 샘플링 레이트는 아날로그 신호를 숫자화 시킨 것


**스펙트럼 (Spectrum)**

* 푸리에 변환을 통해 time domain에서 frequency domain으로 바뀐 그래프
* 특정 시간 길이의 음석 조각 (프레임)이 각각의 주파수 성분들을 얼마칸큼 갖고 있는지 의미

**스펙트로그램 (Spectrogram)**

* 파형 (Waveform)과 스펙트럼 (Spectrum) 조합
* 음향 신호를 주파수, 진폭, 시간으로 구분
* X축 : 시간
* Y축 : 주파수
* Z축 : 진폭



### Zero Crossing Rate


* Compute the zero-crossing rate of an audio time series.
* 신호의 부호가 바뀌는 비율

In [11]:
def get_zero_crossing_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        zero = librosa.feature.zero_crossing_rate(y=y)

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in zero:

            y_feature.append(np.mean(e))

        features.append(y_feature)
    
    zero_df = pd.DataFrame(features,
                           columns=['Zero Crossing Rate'])

    print(zero_df.shape)

    return zero_df

In [12]:
zero_train = get_zero_crossing_feature(df_train)
zero_test = get_zero_crossing_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [13]:
scaler = RobustScaler()
zero_train, zero_test= scaled_df(zero_train,
                                 df_train,
                                 zero_test,
                                 df_test,
                                 scaler)

zero_train.columns = ['Zero_Crossing_Rate']
zero_test.columns = ['Zero_Crossing_Rate']

### RMS

* Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S.

In [14]:
def get_rms_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 RMS 추출
        rms = librosa.feature.rms(y=y)

        y_feature = []
        # 추출된 RMS의 평균을 Feature로 사용
        for e in rms:

            y_feature.append(np.mean(e))

        features.append(y_feature)
    
    rms_df = pd.DataFrame(features,
                           columns=['RMS'])

    print(rms_df.shape)

    return rms_df

In [15]:
rms_train = get_rms_feature(df_train)
rms_test = get_rms_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [16]:
scaler = RobustScaler()
rms_train, rms_test= scaled_df(rms_train,
                                 df_train,
                                 rms_test,
                                 df_test,
                                 scaler)

rms_train.columns = ['RMS']
rms_test.columns = ['RMS']

In [17]:
rms_train.head()

,RMS
0,0.469431
1,-0.054741
2,-0.924593
3,0.749485
4,-0.802393


### Poly Feature

* Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.

In [18]:
def get_poly_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        poly = librosa.feature.poly_features(
                                             y=y,
                                             sr=sr,
                                             #order=2
                                             )

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in poly:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Poly'+str(i) for i in range(len(features[0]))]
    
    poly_df = pd.DataFrame(features,
                           columns=columns)

    print(poly_df.shape)

    return poly_df

In [19]:
poly_train = get_poly_feature(df_train)
poly_test = get_poly_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 2)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 2)


In [20]:
scaler = RobustScaler()
poly_train, poly_test= scaled_df(poly_train,
                                 df_train,
                                 poly_test,
                                 df_test,
                                 scaler)

poly_train.columns = ['Poly_'+str(i) for i in range(len(poly_train.columns))]
poly_test.columns = ['Poly_'+str(i) for i in range(len(poly_test.columns))]

In [21]:
poly_train.head()

,Poly_0,Poly_1
0,-0.493437,0.633444
1,0.177306,-0.483609
2,0.656949,-0.577992
3,-0.524382,0.603702
4,0.722320,-0.496436


### Spectral

#### Spectral Centroid

* Compute the spectral centroid.

* Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins, from which the mean (centroid) is extracted per frame.

* 음성의 각 프레임마다 평균(중심) 주파수를 반환

* Spectrum의 질량중심(Center of mass)을 구한다. Frequency들의 magnitude에 따른 centroid들의 weighted sum이라고 생각하면 된다.

In [22]:
def get_spectral_centroid_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in centroid:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_centroid_'+str(i) for i in range(len(features[0]))]
    
    centroid_df = pd.DataFrame(features,
                           columns=columns)

    print(centroid_df.shape)

    return centroid_df

In [23]:
spectral_centroid_train = get_spectral_centroid_feature(df_train)
spectral_centroid_test = get_spectral_centroid_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [24]:
scaler = RobustScaler()
spectral_centroid_train, spectral_centroid_test= scaled_df(spectral_centroid_train,
                                                           df_train,
                                                           spectral_centroid_test,
                                                           df_test,
                                                           scaler)

spectral_centroid_train.columns = ['Spectral_Centroid_'+str(i) for i in range(len(spectral_centroid_train.columns))]
spectral_centroid_test.columns = ['Spectral_Centroid_'+str(i) for i in range(len(spectral_centroid_test.columns))]

In [25]:
spectral_centroid_train.head()

,Spectral_Centroid_0
0,-0.094233
1,-0.770193
2,0.708246
3,-0.322742
4,1.456958


#### Spectral bandwidth

* Compute p’th-order spectral bandwidth.

* Spectrum의 bandwidth를 측정한다. Frequency들의 magnitude에 따른 centroid의 s.t.d.들의 weighted sum이라고 생각하면 된다.

In [26]:
def get_spectral_bandwidth_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in bandwidth:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_bandwidth_'+str(i) for i in range(len(features[0]))]
    
    bandwidth_df = pd.DataFrame(features,
                           columns=columns)

    print(bandwidth_df.shape)

    return bandwidth_df

In [27]:
spectral_bandwidth_train = get_spectral_bandwidth_feature(df_train)
spectral_bandwidth_test = get_spectral_bandwidth_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [28]:
scaler = RobustScaler()
spectral_bandwidth_train, spectral_bandwidth_test= scaled_df(spectral_bandwidth_train,
                                                           df_train,
                                                           spectral_bandwidth_test,
                                                           df_test,
                                                           scaler)

spectral_bandwidth_train.columns = ['Spectral_Bandwidth_'+str(i) for i in range(len(spectral_bandwidth_train.columns))]
spectral_bandwidth_test.columns = ['Spectral_Bandwidth_'+str(i) for i in range(len(spectral_bandwidth_test.columns))]

In [29]:
spectral_bandwidth_train.head()

,Spectral_Bandwidth_0
0,0.246261
1,-0.529660
2,0.857477
3,0.062208
4,0.008562


#### Spectral contrast

* Compute spectral contrast

* Each frame of a spectrogram S is divided into sub-bands. For each sub-band, the energy contrast is estimated by comparing the mean energy in the top quantile (peak energy) to that of the bottom quantile (valley energy). High contrast values generally correspond to clear, narrow-band signals, while low contrast values correspond to broad-band noise.

In [30]:
def get_spectral_contrast_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in contrast:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_contrast_'+str(i) for i in range(len(features[0]))]
    
    contrast_df = pd.DataFrame(features,
                           columns=columns)

    print(contrast_df.shape)

    return contrast_df

In [31]:
spectral_contrast_train = get_spectral_contrast_feature(df_train)
spectral_contrast_test = get_spectral_contrast_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 7)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 7)


In [32]:
scaler = RobustScaler()
spectral_contrast_train, spectral_contrast_test= scaled_df(spectral_contrast_train,
                                                           df_train,
                                                           spectral_contrast_test,
                                                           df_test,
                                                           scaler)

spectral_contrast_train.columns = ['Spectral_Contrast_'+str(i) for i in range(len(spectral_contrast_train.columns))]
spectral_contrast_test.columns = ['Spectral_Contrast_'+str(i) for i in range(len(spectral_contrast_test.columns))]

In [33]:
spectral_contrast_train.head()

,Spectral_Contrast_0,Spectral_Contrast_1,Spectral_Contrast_2,Spectral_Contrast_3,Spectral_Contrast_4,Spectral_Contrast_5,Spectral_Contrast_6
0,-0.429224,0.327507,0.043187,-0.201558,-0.255015,-0.487467,-0.806537
1,-0.181066,-0.629994,0.587698,0.194772,0.606893,0.411807,-0.431408
2,0.340532,3.217004,1.850249,-0.072913,0.300461,-0.167747,0.736412
3,-0.053227,-0.895404,1.922328,0.658625,0.297013,-0.104181,-0.190854
4,-0.759927,0.440922,-1.401846,0.634863,0.265792,0.130915,1.954896


#### Spectral Flatness

* Compute spectral flatness

* Spectral flatness (or tonality coefficient) is a measure to quantify how much noise-like a sound is, as opposed to being tone-like 1. A high spectral flatness (closer to 1.0) indicates the spectrum is similar to white noise. It is often converted to decibel.

* Spectrum의 noisiness (또는 tonality)를 구한다. Frequency들의 기하평균'(geometric mean)에 대한 산술평균(arithmetic mean)의 비를 이용해 구할 수 있다. 1에 가까울수록 white noise (maximum flatness)를 의미한다.

In [34]:
def get_spectral_flatness_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        flatness = librosa.feature.spectral_flatness(y=y)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in flatness:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_flatness_'+str(i) for i in range(len(features[0]))]
    
    flatness_df = pd.DataFrame(features,
                           columns=columns)

    print(flatness_df.shape)

    return flatness_df

In [35]:
spectral_flatness_train = get_spectral_flatness_feature(df_train)
spectral_flatness_test = get_spectral_flatness_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [36]:
scaler = RobustScaler()
spectral_flatness_train, spectral_flatness_test= scaled_df(spectral_flatness_train,
                                                           df_train,
                                                           spectral_flatness_test,
                                                           df_test,
                                                           scaler)

spectral_flatness_train.columns = ['Spectral_Flatness_'+str(i) for i in range(len(spectral_flatness_train.columns))]
spectral_flatness_test.columns = ['Spectral_Flatness_'+str(i) for i in range(len(spectral_flatness_test.columns))]

In [37]:
spectral_flatness_train.head()

,Spectral_Flatness_0
0,0.099881
1,-0.722606
2,0.821307
3,-0.389390
4,1.597347


#### Spectral Rolloff

* Compute roll-off frequency.

* The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and the bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent to a value close to 1 (or 0).

* 스펙트로그렘에서 roll percent 위치에 차지하는 주파수를 구함

In [38]:
def get_spectral_rolloff_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        rolloff = librosa.feature.spectral_rolloff(y=y,
                                                   #roll_percent=0.85
                                                   )

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in rolloff:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_rolloff_'+str(i) for i in range(len(features[0]))]
    
    rolloff_df = pd.DataFrame(features,
                           columns=columns)

    print(rolloff_df.shape)

    return rolloff_df

In [39]:
spectral_rolloff_train = get_spectral_rolloff_feature(df_train)
spectral_rolloff_test = get_spectral_rolloff_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [40]:
scaler = RobustScaler()
spectral_rolloff_train, spectral_rolloff_test= scaled_df(spectral_rolloff_train,
                                                           df_train,
                                                           spectral_rolloff_test,
                                                           df_test,
                                                           scaler)

spectral_rolloff_train.columns = ['Spectral_Rolloff_'+str(i) for i in range(len(spectral_rolloff_train.columns))]
spectral_rolloff_test.columns = ['Spectral_Rolloff_'+str(i) for i in range(len(spectral_rolloff_test.columns))]

In [41]:
spectral_rolloff_train.head()

,Spectral_Rolloff_0
0,-0.055766
1,-0.905437
2,1.298731
3,-0.256522
4,1.197487


### Mel Spectrogram

* 사람은 낮은 주파수를 높은 주파수보다 더 예민하게 받아들임

* 500 ~ 1500 Hz 가 바뀌는건 예민하게 인식하는데 반해 10000Hz ~ 11000Hz가 바뀌는 것은 잘 인식 못한다는 것.

* 주파수 단위를 다음 공식에 따라 멜 스케일로 변환 m = 2595 \log_{10}\left(1 + \frac{f}{700}\right)m=2595log10​(1+700f​)

* 해당 scale을 적용해 spectogram을 만든 것이 melspectogram

In [42]:
def get_melspectrogram_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 melspectrogram 추출
        melspectrogram = librosa.feature.melspectrogram(y=y,
                                                        sr=sr,
                                                        n_fft=2048,
                                                        hop_length=512,
                                                        )

        y_feature = []
        # 추출된 melspectrogram들의 평균을 Feature로 사용
        for e in melspectrogram:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Mel_Spectrogram_'+str(i) for i in range(len(features[0]))]
    
    melspectrogram_df = pd.DataFrame(features,
                           columns=columns)

    print(melspectrogram_df.shape)

    return melspectrogram_df

In [43]:
melspectrogram_train = get_melspectrogram_feature(df_train)
melspectrogram_test = get_melspectrogram_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 128)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 128)


In [44]:
scaler = RobustScaler()
melspectrogram_train, melspectrogram_test= scaled_df(melspectrogram_train,
                                       df_train,
                                       melspectrogram_test,
                                       df_test,
                                       scaler)

melspectrogram_train.columns = ['Mel_Spectrogram_'+str(i) for i in range(len(melspectrogram_train.columns))]
melspectrogram_test.columns = ['Mel_Spectrogram_'+str(i) for i in range(len(melspectrogram_test.columns))]

In [45]:
melspectrogram_train.head()

,Mel_Spectrogram_0,Mel_Spectrogram_1,Mel_Spectrogram_2,Mel_Spectrogram_3,Mel_Spectrogram_4,Mel_Spectrogram_5,Mel_Spectrogram_6,Mel_Spectrogram_7,Mel_Spectrogram_8,Mel_Spectrogram_9,...,Mel_Spectrogram_118,Mel_Spectrogram_119,Mel_Spectrogram_120,Mel_Spectrogram_121,Mel_Spectrogram_122,Mel_Spectrogram_123,Mel_Spectrogram_124,Mel_Spectrogram_125,Mel_Spectrogram_126,Mel_Spectrogram_127
0,0.058931,0.476081,0.113896,0.374052,1.966384,0.994531,0.856271,0.587199,0.868910,0.296706,...,0.369954,-0.066142,-0.236345,0.250120,0.013667,0.097357,0.290392,0.184182,0.392811,-0.141753
1,1.287636,0.633630,-0.432464,1.620988,0.484846,0.681291,0.498891,-0.312905,-0.230217,-0.291431,...,-0.470382,-0.268224,-0.303994,-0.358652,-0.415375,-0.297280,-0.241743,-0.191959,-0.229430,-0.335791
2,-0.829404,-1.158249,-0.978606,-0.534312,-0.177732,0.986787,-0.187644,-0.325331,-0.493828,-0.636806,...,0.860542,1.108199,0.717456,0.857656,0.730048,1.197507,1.063653,0.990560,1.177343,0.979368
3,0.614694,0.997716,0.470939,0.722892,1.750515,0.684190,0.285374,1.306666,0.436435,0.100710,...,-0.097282,0.066587,0.410269,-0.190445,-0.092478,0.106614,-0.292512,-0.292223,-0.015245,0.510250
4,-0.729482,-0.750588,-0.555883,-0.841748,-1.475900,-1.240271,-1.105781,-0.542818,-0.548657,-0.649213,...,0.525957,1.112205,1.375385,0.402423,0.317616,1.067398,0.647372,0.220570,0.407480,0.794141


### MFCC

* Mel-Spectrogram이라는 피쳐에 대해 행렬을 압축해서 표현해주는 DCT 연산을 수행

In [46]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y,
                                    sr=sr,
                                    n_mfcc=128,
                                    dct_type=2)

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['MFCC_'+str(i) for i in range(len(features[0]))]
    
    mfcc_df = pd.DataFrame(features,
                           columns=columns)

    print(mfcc_df.shape)

    return mfcc_df

In [47]:
mfcc_train = get_mfcc_feature(df_train)
mfcc_test = get_mfcc_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 128)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 128)


In [48]:
scaler = RobustScaler()
mfcc_train, mfcc_test= scaled_df(mfcc_train,
                                 df_train,
                                 mfcc_test,
                                 df_test,
                                 scaler)

mfcc_train.columns = ['MFCC_'+str(i) for i in range(len(mfcc_train.columns))]
mfcc_test.columns = ['MFCC_'+str(i) for i in range(len(mfcc_test.columns))]

In [49]:
mfcc_train.head()

,MFCC_0,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,...,MFCC_118,MFCC_119,MFCC_120,MFCC_121,MFCC_122,MFCC_123,MFCC_124,MFCC_125,MFCC_126,MFCC_127
0,0.690073,0.114611,0.123688,0.071540,0.521380,0.803779,0.856026,-0.493195,-0.880081,0.120076,...,1.228098,1.704697,1.202587,-0.981697,-1.002171,-0.184173,1.812308,-0.012843,-0.669357,-1.577192
1,-1.703092,0.668559,-0.234100,0.031342,0.113181,0.243570,-0.116445,-0.638444,0.364353,-0.605461,...,-0.253236,-0.944080,-0.756294,-0.038057,0.697295,0.526572,0.057620,-0.350332,0.215434,0.008132
2,0.376408,-0.810147,1.061598,-0.635059,-0.387792,-0.655307,-0.271630,-2.172539,3.149782,-0.484744,...,0.523302,-0.275989,-0.360429,-0.682915,0.169130,0.826519,0.255326,-0.505536,-0.829216,-1.125630
3,0.322113,0.314324,0.464049,0.119507,0.319502,0.547200,1.188367,0.886476,0.742252,0.245832,...,-0.144778,1.086816,-0.164982,0.154664,-0.448467,-0.608437,-0.181108,-0.003359,-1.061261,0.141075
4,0.576171,-1.679476,-2.268337,-1.221650,-1.985492,-1.582854,-1.859410,-1.218810,-1.348051,-1.465109,...,-0.225962,0.116558,-0.203816,0.842311,-0.367322,-0.961788,-0.447271,-0.070059,-0.111112,-0.029130


### Chroma

* 모든 음표는 12개의 pitch(음높이)와 octave로 구성되어 있다.

* Chroma는 각 audio frame의 octave는 무시하고, 12개의 pitch class에 대한 분포를 나타낸 것이다.

* Pitch class는 C, C#, D, D#, E, F, F#, G, G#, A, A#, B로 구성되어 있다.

#### Chroma stft

* Compute a chromagram from a waveform or power spectrogram.

In [50]:
def get_chroma_stft_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_stft(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_stft_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [51]:
chroma_stft_train = get_chroma_stft_feature(df_train)
chroma_stft_test = get_chroma_stft_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 12)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 12)


In [52]:
scaler = RobustScaler()
chroma_stft_train, chroma_stft_test= scaled_df(chroma_stft_train,
                                     df_train,
                                     chroma_stft_test,
                                     df_test,
                                     scaler)

chroma_stft_train.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_train.columns))]
chroma_stft_test.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_test.columns))]

In [53]:
chroma_stft_train.head()

,Chroma_stft_0,Chroma_stft_1,Chroma_stft_2,Chroma_stft_3,Chroma_stft_4,Chroma_stft_5,Chroma_stft_6,Chroma_stft_7,Chroma_stft_8,Chroma_stft_9,Chroma_stft_10,Chroma_stft_11
0,-0.591707,0.571112,1.823861,1.820528,-0.541722,-1.457691,1.349097,1.380352,0.202552,-1.036793,0.378324,-0.043309
1,-0.557744,-0.312736,0.296492,0.442020,-0.098537,0.014428,-0.322188,-0.505186,-0.322056,-0.174136,0.359876,-0.171708
2,-0.921887,-1.317806,-0.748789,-0.449534,4.253784,5.819659,1.060681,-0.940750,-1.017104,-1.129097,-1.299533,-1.226438
3,-0.637213,-0.732822,-0.922129,-0.863983,0.689097,0.237948,-0.373268,-0.725025,-0.496817,0.005216,-0.984049,-0.610303
4,0.253212,-0.533289,-0.678308,-0.830982,-0.421682,0.391867,0.609555,0.684925,1.623016,0.803613,0.557382,0.452940


#### Chroma cqt

* Constant-Q chromagram

In [54]:
def get_chroma_cpt_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_cqt(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_cqt_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [55]:
chroma_cqt_train = get_chroma_cpt_feature(df_train)
chroma_cqt_test = get_chroma_cpt_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 12)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 12)


In [56]:
scaler = RobustScaler()
chroma_cqt_train, chroma_cqt_test= scaled_df(chroma_cqt_train,
                                     df_train,
                                     chroma_cqt_test,
                                     df_test,
                                     scaler)

chroma_cqt_train.columns = ['Chroma_cqt_'+str(i) for i in range(len(chroma_cqt_train.columns))]
chroma_cqt_test.columns = ['Chroma_cqt_'+str(i) for i in range(len(chroma_cqt_test.columns))]

In [57]:
chroma_cqt_train.head()

,Chroma_cqt_0,Chroma_cqt_1,Chroma_cqt_2,Chroma_cqt_3,Chroma_cqt_4,Chroma_cqt_5,Chroma_cqt_6,Chroma_cqt_7,Chroma_cqt_8,Chroma_cqt_9,Chroma_cqt_10,Chroma_cqt_11
0,1.417874,-0.126394,0.197389,1.079055,0.112661,0.034992,0.610548,1.802173,-0.123113,0.662423,0.739431,1.299744
1,-0.613785,0.116286,0.101423,0.736887,1.462314,1.183265,0.467960,0.206767,0.722733,1.000517,1.181474,0.379425
2,-1.021166,0.297485,-0.086821,-0.344791,1.265631,0.774997,0.257090,-0.813932,-0.222772,-0.222696,-0.533417,0.410030
3,1.126840,-0.262245,0.349518,-0.592407,0.102506,0.788705,-0.008489,-0.809610,-0.999229,0.160308,0.117831,0.565213
4,-0.081414,-0.487453,-0.361974,0.071876,0.540712,-0.459921,1.195416,0.820027,1.218448,-0.024421,0.056718,0.110269


#### Chroma cens

* Computes the chroma variant “Chroma Energy Normalized” (CENS)

* To compute CENS features, following steps are taken after obtaining chroma vectors using chroma_cqt: 1.

  - L-1 normalization of each chroma vector

  - Quantization of amplitude based on “log-like” amplitude thresholds

  - (optional) Smoothing with sliding window. Default window length = 41 frames

  - (not implemented) Downsampling

* CENS features are robust to dynamics, timbre and articulation, thus these are commonly used in audio matching and retrieval applications.

In [58]:
def get_chroma_cens_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_cens(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_cens_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [59]:
chroma_cens_train = get_chroma_cens_feature(df_train)
chroma_cens_test = get_chroma_cens_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 12)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 12)


In [60]:
scaler = RobustScaler()
chroma_cens_train, chroma_cens_test= scaled_df(chroma_cens_train,
                                     df_train,
                                     chroma_cens_test,
                                     df_test,
                                     scaler)

chroma_cens_train.columns = ['Chroma_cens_'+str(i) for i in range(len(chroma_cens_train.columns))]
chroma_cens_test.columns = ['Chroma_cens_'+str(i) for i in range(len(chroma_cens_test.columns))]

### Tonnetz

* Computes the tonal centroid features (tonnetz)

* This representation uses the method of to project chroma features onto a 6-dimensional basis representing the perfect fifth, minor third, and major third each as two-dimensional coordinates.

* 1739년 오일러에 의해 처음 기술된 것으로 음악의 tonality와 tonal space의 관계를 graphical한 방식으로 표현한 것

In [61]:
def get_tonnetz_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)

        # librosa패키지를 사용하여 chroma 추출
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12)
        #chroma = librosa.feature.chroma_cqt(y=y, sr=sr, n_chroma=12)
        #chroma = librosa.feature.chroma_cens(y=y, sr=sr, n_chroma=12)
        
        # librosa패키지를 사용하여 tonnetz 추출
        tonnetz = librosa.feature.tonnetz(y=y, sr=sr,
                                          n_chroma=12,
                                          chroma=chroma)

        y_feature = []
        # 추출된 tonnetz들의 평균을 Feature로 사용
        for e in tonnetz:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Tonnetz_'+str(i) for i in range(len(features[0]))]
    
    tonnetz_df = pd.DataFrame(features,
                           columns=columns)

    print(tonnetz_df.shape)

    return tonnetz_df

In [62]:
tonnetz_train = get_tonnetz_feature(df_train)
tonnetz_test = get_tonnetz_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 6)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 6)


In [63]:
scaler = RobustScaler()
tonnetz_train, tonnetz_test= scaled_df(tonnetz_train,
                                       df_train,
                                       tonnetz_test,
                                       df_test,
                                       scaler)

tonnetz_train.columns = ['Tonnetz_'+str(i) for i in range(len(tonnetz_train.columns))]
tonnetz_test.columns = ['Tonnetz_'+str(i) for i in range(len(tonnetz_test.columns))]

In [64]:
tonnetz_train.head()

,Tonnetz_0,Tonnetz_1,Tonnetz_2,Tonnetz_3,Tonnetz_4,Tonnetz_5
0,-0.549556,-0.400232,1.600160,-1.338966,0.486196,-0.167790
1,-0.244280,-0.153516,0.457955,-0.670473,-0.626542,-0.301937
2,0.917282,0.671949,-2.884510,1.660699,-0.224340,-1.346087
3,1.003935,-0.457939,-0.340745,0.948281,0.063535,-0.268656
4,-0.337871,0.458990,-0.082489,0.430681,-0.794878,0.049528


### Rhythm

* Compute the tempogram: local autocorrelation of the onset strength envelope.

In [65]:
def get_rhythm_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)

        # librosa패키지를 사용하여 onset 추출
        onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
        #onset_envelope = librosa.onset.onset_detect(y=y, sr=sr)
        #onset_envelope = librosa.onset.onset_strength_multi(y=y, sr=sr)
        
        # librosa패키지를 사용하여 Rhythm 추출
        rhythm = librosa.feature.tempogram(y=y, sr=sr,
                                           onset_envelope=onset_envelope)

        y_feature = []
        # 추출된 tonnetz들의 평균을 Feature로 사용
        for e in rhythm:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Rhythm'+str(i) for i in range(len(features[0]))]
    
    rhythm_df = pd.DataFrame(features,
                           columns=columns)

    print(rhythm_df.shape)

    return rhythm_df

In [66]:
rhythm_train = get_rhythm_feature(df_train)
rhythm_test = get_rhythm_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 384)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 384)


In [67]:
scaler = RobustScaler()
rhythm_train, rhythm_test= scaled_df(rhythm_train,
                                       df_train,
                                       rhythm_test,
                                       df_test,
                                       scaler)

rhythm_train.columns = ['Rhythm_'+str(i) for i in range(len(rhythm_train.columns))]
rhythm_test.columns = ['Rhythm_'+str(i) for i in range(len(rhythm_test.columns))]

In [68]:
rhythm_train.head()

,Rhythm_0,Rhythm_1,Rhythm_2,Rhythm_3,Rhythm_4,Rhythm_5,Rhythm_6,Rhythm_7,Rhythm_8,Rhythm_9,...,Rhythm_374,Rhythm_375,Rhythm_376,Rhythm_377,Rhythm_378,Rhythm_379,Rhythm_380,Rhythm_381,Rhythm_382,Rhythm_383
0,0.0,-0.551812,-0.544812,-0.230067,-0.241238,-0.306453,0.512676,-0.025339,-0.379115,0.468742,...,0.814667,0.814972,0.815410,0.816025,0.816716,0.817522,0.818483,0.819925,0.820260,-1.678619e-17
1,0.0,-0.114550,-0.142752,-0.471972,-0.364540,-0.330001,-0.179695,0.018062,-0.075052,-0.536748,...,-0.018767,-0.018371,-0.017928,-0.017456,-0.016750,-0.015348,-0.013955,-0.012485,-0.011624,2.535897e-18
2,0.0,0.306098,0.272140,0.127627,0.259484,0.731201,-0.294582,0.253361,0.954066,-0.011677,...,-0.138610,-0.138542,-0.138471,-0.138459,-0.138247,-0.137380,-0.136617,-0.135886,-0.135844,-3.204918e-18
3,0.0,-0.616099,-0.796693,-0.594651,0.134939,-1.123321,-0.998410,0.557776,0.151244,-0.746797,...,0.257778,0.257546,0.257318,0.257110,0.256893,0.256684,0.256491,0.256524,0.256368,1.121183e-17
4,0.0,-0.501435,-1.155440,-0.765027,0.321320,0.894521,-0.430990,-0.844253,-0.609254,0.423174,...,0.535515,0.534290,0.533128,0.532047,0.530947,0.529756,0.528410,0.527208,0.525431,-9.377353e-18


## Dimension Reduction

### Truncated SVD

### PCA

In [69]:
pca = PCA()
pca.fit(melspectrogram_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

100


In [70]:
pca = PCA(n_components=N_COMPONETS)

pca_train_mel = pca.fit_transform(melspectrogram_train)
pca_test_mel = pca.transform(melspectrogram_test)

pca_train_mel = pd.DataFrame(pca_train_mel)
pca_test_mel = pd.DataFrame(pca_test_mel)

### Rhythm

In [97]:
pca = PCA()
pca.fit(rhythm_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

119


In [98]:
pca = PCA(n_components=N_COMPONETS)

pca_train_rhythm = pca.fit_transform(rhythm_train)
pca_test_rhythm = pca.transform(rhythm_test)

pca_train_rhythm = pd.DataFrame(pca_train_rhythm)
pca_test_rhythm = pd.DataFrame(pca_test_rhythm)

### Sparse PCA

#### Zero Crossing Rate

In [71]:
pca = PCA()
pca.fit(zero_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [72]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_zero = pca.fit_transform(zero_train)
pca_test_zero = pca.transform(zero_test)

pca_train_zero = pd.DataFrame(pca_train_zero)
pca_test_zero = pd.DataFrame(pca_test_zero)

#### RMS

In [73]:
pca = PCA()
pca.fit(rms_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [74]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_rms = pca.fit_transform(rms_train)
pca_test_rms = pca.transform(rms_test)

pca_train_rms = pd.DataFrame(pca_train_rms)
pca_test_rms = pd.DataFrame(pca_test_rms)

#### MFCC

In [75]:
pca = PCA()
pca.fit(mfcc_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

124


In [76]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_mfcc = pca.fit_transform(mfcc_train)
pca_test_mfcc = pca.transform(mfcc_test)

pca_train_mfcc = pd.DataFrame(pca_train_mfcc)
pca_test_mfcc = pd.DataFrame(pca_test_mfcc)

#### Spectral


##### Spectral Centroid

In [77]:
pca = PCA()
pca.fit(spectral_centroid_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [78]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_centroid = pca.fit_transform(spectral_centroid_train)
pca_test_sepctral_centroid = pca.transform(spectral_centroid_test)

pca_train_sepctral_centroid = pd.DataFrame(pca_train_sepctral_centroid)
pca_test_sepctral_centroid = pd.DataFrame(pca_test_sepctral_centroid)

##### Spectral Bandwidth

In [79]:
pca = PCA()
pca.fit(spectral_bandwidth_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [80]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_bandwidth = pca.fit_transform(spectral_bandwidth_train)
pca_test_sepctral_bandwidth = pca.transform(spectral_bandwidth_test)

pca_train_sepctral_bandwidth = pd.DataFrame(pca_train_sepctral_bandwidth)
pca_test_sepctral_bandwidth = pd.DataFrame(pca_test_sepctral_bandwidth)

##### Spectral Contrast

In [81]:
pca = PCA()
pca.fit(spectral_contrast_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

7


In [82]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_contrast = pca.fit_transform(spectral_contrast_train)
pca_test_sepctral_contrast = pca.transform(spectral_contrast_test)

pca_train_sepctral_contrast = pd.DataFrame(pca_train_sepctral_contrast)
pca_test_sepctral_contrast = pd.DataFrame(pca_test_sepctral_contrast)

##### Spectral Flatness


In [83]:
pca = PCA()
pca.fit(spectral_flatness_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [84]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_flatness = pca.fit_transform(spectral_flatness_train)
pca_test_sepctral_flatness = pca.transform(spectral_flatness_test)

pca_train_sepctral_flatness = pd.DataFrame(pca_train_sepctral_flatness)
pca_test_sepctral_flatness = pd.DataFrame(pca_test_sepctral_flatness)

##### Spectral Rolloff

In [85]:
pca = PCA()
pca.fit(spectral_rolloff_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [86]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_rolloff = pca.fit_transform(spectral_rolloff_train)
pca_test_sepctral_rolloff = pca.transform(spectral_rolloff_test)

pca_train_sepctral_rolloff = pd.DataFrame(pca_train_sepctral_rolloff)
pca_test_sepctral_rolloff = pd.DataFrame(pca_test_sepctral_rolloff)

#### Chroma

##### Chroma stft

In [87]:
pca = PCA()
pca.fit(chroma_stft_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

12


In [88]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_chroma_stft = pca.fit_transform(chroma_stft_train)
pca_test_chroma_stft = pca.transform(chroma_stft_test)

pca_train_chroma_stft = pd.DataFrame(pca_train_chroma_stft)
pca_test_chroma_stft = pd.DataFrame(pca_test_chroma_stft)

##### Chroma cqt

In [89]:
pca = PCA()
pca.fit(chroma_cqt_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

12


In [90]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_chroma_cqt = pca.fit_transform(chroma_cqt_train)
pca_test_chroma_cqt = pca.transform(chroma_cqt_test)

pca_train_chroma_cqt = pd.DataFrame(pca_train_chroma_cqt)
pca_test_chroma_cqt = pd.DataFrame(pca_test_chroma_cqt)

##### Chroma cens

In [91]:
pca = PCA()
pca.fit(chroma_cens_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

12


In [92]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_chroma_cens = pca.fit_transform(chroma_cens_train)
pca_test_chroma_cens = pca.transform(chroma_cens_test)

pca_train_chroma_cens = pd.DataFrame(pca_train_chroma_cens)
pca_test_chroma_cens = pd.DataFrame(pca_test_chroma_cens)

### Tonnetz

In [93]:
pca = PCA()
pca.fit(tonnetz_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

6


In [94]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_tonnetz = pca.fit_transform(tonnetz_train)
pca_test_tonnetz = pca.transform(tonnetz_test)

pca_train_tonnetz = pd.DataFrame(pca_train_tonnetz)
pca_test_tonnetz = pd.DataFrame(pca_test_tonnetz)

### Kernel PCA

#### Poly Feature

In [99]:
pca = PCA()
pca.fit(poly_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

2


In [100]:
pca = KernelPCA(n_components=N_COMPONETS)

pca_train_poly = pca.fit_transform(poly_train)
pca_test_poly = pca.transform(poly_test)

pca_train_poly = pd.DataFrame(pca_train_poly)
pca_test_poly = pd.DataFrame(pca_test_poly)

# Concat Preprocessed Data Set

In [608]:
preprocessed_train_mean = pd.concat([
                                     #pca_train_mel,
                                     pca_train_zero,
                                     pca_train_rms,
                                     pca_train_mfcc,
                                     #pca_train_sepctral_centroid,
                                     #pca_train_sepctral_bandwidth,
                                     #pca_train_sepctral_contrast,
                                     #pca_train_sepctral_flatness,
                                     #pca_train_sepctral_rolloff,
                                     #pca_train_chroma_stft,
                                     #pca_train_chroma_cqt,
                                     #pca_train_chroma_cens,
                                     #pca_train_tonnetz,
                                     #pca_train_rhythm,
                                     #pca_train_poly
                                    ], axis=1)

preprocessed_test_mean = pd.concat([
                                     #pca_test_mel,
                                     pca_test_zero,
                                     pca_test_rms,
                                     pca_test_mfcc,
                                     #pca_test_sepctral_centroid,
                                     #pca_test_sepctral_bandwidth,
                                     #pca_test_sepctral_contrast,
                                     #pca_test_sepctral_flatness,
                                     #pca_test_sepctral_rolloff,
                                     #pca_test_chroma_stft,
                                     #pca_test_chroma_cqt,
                                     #pca_test_chroma_cens,
                                     #pca_test_tonnetz,
                                     #pca_test_rhythm,
                                     #pca_test_poly
                                    ], axis=1)

In [609]:
preprocessed_train_mean.columns = [i for i in range(len(preprocessed_train_mean.columns))]
preprocessed_test_mean.columns = [i for i in range(len(preprocessed_test_mean.columns))]

In [610]:
print(preprocessed_train_mean.shape)
preprocessed_train_mean.head()

(1279, 126)


,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,-0.187735,0.506005,-1.981614,0.171828,-2.260467,1.094061,-4.093776,2.614363,-0.710745,6.390842,...,0.001501,-0.127217,0.064390,0.006651,-0.144934,-0.138402,0.066886,-0.080058,0.019512,0.116428
1,-0.562140,-0.012977,-1.344541,1.281452,-0.623583,0.060917,1.820375,0.294443,0.665423,0.109686,...,0.042284,0.066112,0.072370,0.122463,0.137039,0.053279,0.133139,0.166125,-0.445497,-0.017889
2,0.280977,-0.874217,1.814506,1.315035,0.464657,0.152956,-2.995696,2.591512,1.804725,-1.511827,...,-0.424342,-0.079264,-0.008926,0.174747,-0.194866,0.369738,0.223598,0.051028,-0.115109,0.274387
3,-0.551061,0.783286,0.091396,0.872244,0.155946,0.019353,0.239218,3.061080,-0.012733,0.740089,...,-0.187094,-0.002030,0.023713,-0.101980,-0.150240,-0.093934,0.272524,-0.002639,-0.158889,-0.169270
4,1.210362,-0.753227,0.861370,0.136026,0.322987,1.740404,2.504216,-3.772065,-1.675104,-0.643610,...,0.023425,-0.566661,-0.194356,0.189694,0.192459,-0.293593,0.150898,0.177200,-0.005592,0.190851


# Modeling

## LOF

### 모델 생성

In [612]:
model = LocalOutlierFactor(n_neighbors=2,
                           algorithm='auto',
                           contamination='auto',
                           novelty=True)

### 모델 학습

In [613]:
model.fit(preprocessed_train_mean)

LocalOutlierFactor(n_neighbors=2, novelty=True)

### 추론

In [614]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량) 이므로 (0:정상, 1:불량)로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [615]:
test_pred = model.predict(preprocessed_test_mean) 
test_pred = get_pred_label(test_pred)

## Isolate Forest

### 모델 생성

In [544]:
model = IsolationForest(n_estimators=200,
                        max_samples=256,
                        contamination='auto',
                        random_state=42,
                        verbose=1)

### 모델 학습

In [545]:
model.fit(preprocessed_train_mean)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


IsolationForest(max_samples=256, n_estimators=200, random_state=42, verbose=1)

### 추론

In [558]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량) 이므로 (0:정상, 1:불량)로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [547]:
test_pred = model.predict(preprocessed_test_mean) # model prediction
test_pred = get_pred_label(test_pred)

# Submission

In [616]:
submit = pd.read_csv('./sample_submission.csv')

In [617]:
submit['LABEL'] = test_pred
submit.head()

,SAMPLE_ID,LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,1


In [618]:
submit['LABEL'].value_counts()

1    794
0    720
Name: LABEL, dtype: int64

In [580]:
submit.to_csv('./submit60.csv', index=False)